<a href="https://colab.research.google.com/github/graceboatman/COVID-19-Research/blob/master/COVID_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pytrends

from google.colab import drive
import pandas as pd
import numpy as np
from pytrends.request import TrendReq
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn import preprocessing

# Mount google drive
DRIVE_MOUNT='/content/gdrive'
drive.mount(DRIVE_MOUNT)

pytrend = TrendReq()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# State Level Data

In [0]:
# GOOGLE TRENDS DATA

# list of inputs to 'kw_list'
keywords = [['isolation', 'exclusion', 'left out', 'ostracism', 'lonely'], 
            ['loneliness', 'alone', 'depression', 'anxiety', 'bored'], 
            ['overwhelmed', 'angry', 'frustration', 'missing friends', 'stay in touch'], 
            ['worry', 'going out', 'go outside', 'self teach', 'hopeless'], 
            ['remote', 'segregated', 'discriminative', 'abandoned', 'tired'], 
            ['confused', 'separated', 'distant', 'disconnected', 'sad'], 
            ['stressed']]

# list of inputs to 'geo' field
def us(string):
    return 'US-' + string

states = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')
state_name = list(states['State'])
abrev = states['Abbreviation']
abrev = list(abrev.map(us))

# input to 'timeframe'
timeframe = '2020-01-21 2020-06-03'

for i, state_abrev in enumerate(abrev):
    for j, kw_list in enumerate(keywords):
        pytrend.build_payload(kw_list, geo = state_abrev, timeframe = timeframe)
        if j == 0:
            agg_df = pytrend.interest_over_time().drop(columns=['isPartial'])
        else:
            df = pytrend.interest_over_time().drop(columns=['isPartial'])
            agg_df = pd.merge(agg_df, df, on='date')      
    agg_df.insert(0, 'state', state_name[i])
    agg_df = agg_df.reset_index()

    if i == 0:
        all_df = agg_df
    else:
        all_df = all_df.append(agg_df)
all_df

#all_df.to_csv('gdrive/My Drive/COVID-19/states_daily.csv')

In [0]:
# COVID-19 DATA

state_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
state_df['date'] = pd.to_datetime(state_df['date'])

state_df = state_df.sort_values(by = ['state','date'])
state_df = state_df.reset_index().drop(columns = ['index'])

# get daily increase values
daily_increase_cases = []
daily_increase_deaths = []

for index, row in state_df.iterrows():
    if index == 0:
        new_cases = row['cases']
        daily_increase_cases.append(new_cases)
        # new deaths
        new_deaths = row['deaths']
        daily_increase_deaths.append(new_deaths)

    elif row['state'] == prev['state']:
        new_cases = (row['cases'] - prev['cases'])/(row['date'] - prev['date']).days
        daily_increase_cases.append(new_cases)
        # new deaths
        new_deaths = (row['deaths'] - prev['deaths'])/(row['date'] - prev['date']).days
        daily_increase_deaths.append(new_deaths)
    else:
        new_cases = row['cases']
        daily_increase_cases.append(new_cases)
        # new deaths
        new_deaths = row['deaths']
        daily_increase_deaths.append(new_deaths)
    
    prev = row

state_df['cases_daily'] = daily_increase_cases
state_df['deaths_daily'] = daily_increase_deaths

state_df


,date,state,fips,cases,deaths,cases_daily,deaths_daily
0,2020-03-13,Alabama,1,6,0,6.0,0.0
1,2020-03-14,Alabama,1,12,0,6.0,0.0
2,2020-03-15,Alabama,1,23,0,11.0,0.0
3,2020-03-16,Alabama,1,29,0,6.0,0.0
4,2020-03-17,Alabama,1,39,0,10.0,0.0
...,...,...,...,...,...,...,...
5069,2020-05-29,Wyoming,56,891,15,15.0,0.0
5070,2020-05-30,Wyoming,56,898,16,7.0,1.0
5071,2020-05-31,Wyoming,56,903,16,5.0,0.0
5072,2020-06-01,Wyoming,56,910,17,7.0,1.0


In [0]:
# MERGE GOOGLE TRENDS AND COVID_19 DATA

#all_df = pd.read_csv('gdrive/My Drive/COVID-19/states_daily.csv').drop(columns = ['Unnamed: 0'])

#state_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv')
#state_df['date'] = pd.to_datetime(state_df['date'])

#merged_df = pd.merge(all_df, state_df, on = ['state','date'])
#merged_df.to_csv('gdrive/My Drive/COVID-19/states_daily_merged.csv')

merged_df = pd.read_csv('gdrive/My Drive/COVID-19/states_daily_merged.csv').drop(columns = 'Unnamed: 0')
merged_df

,date,state,isolation,exclusion,left out,ostracism,lonely,loneliness,alone,depression,anxiety,bored,overwhelmed,angry,frustration,missing friends,stay in touch,worry,going out,go outside,self teach,hopeless,remote,segregated,discriminative,abandoned,tired,confused,separated,distant,disconnected,sad,stressed,fips,cases,deaths,cases_daily,deaths_daily
0,2020-03-13,Alabama,0,21,21,0,32,0,31,46,57,15,14,48,0,0,0,0,15,0,0,0,58,0,0,5,18,14,14,0,14,100,35,1,6,0,6.0,0.0
1,2020-03-14,Alabama,64,0,25,0,64,0,34,31,44,9,0,69,0,0,0,53,18,0,0,0,62,0,6,8,13,17,17,0,0,69,0,1,12,0,6.0,0.0
2,2020-03-15,Alabama,62,0,49,0,37,0,45,36,51,27,17,100,0,0,0,34,35,17,0,17,66,0,0,18,25,0,17,17,0,61,0,1,23,0,11.0,0.0
3,2020-03-16,Alabama,53,0,21,0,32,0,29,39,36,35,0,48,0,0,0,22,52,15,0,0,64,0,0,7,25,0,14,29,0,67,0,1,29,0,6.0,0.0
4,2020-03-17,Alabama,43,22,32,0,0,0,45,61,42,18,15,39,15,0,0,0,22,0,0,0,75,0,0,8,11,0,0,29,15,53,0,1,39,0,10.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4657,2020-05-27,Wyoming,0,0,0,0,0,0,22,0,34,22,0,0,0,0,0,0,0,43,0,0,37,0,0,28,28,28,0,0,0,27,0,56,860,14,10.0,1.0
4658,2020-05-28,Wyoming,0,0,0,0,0,23,45,23,45,0,0,84,0,0,0,45,89,0,0,0,42,0,0,0,0,0,0,0,0,28,0,56,876,15,16.0,1.0
4659,2020-05-29,Wyoming,0,0,0,0,38,0,24,48,24,0,0,0,43,0,0,0,0,0,0,0,72,0,0,29,28,0,0,0,0,29,0,56,891,15,15.0,0.0
4660,2020-05-30,Wyoming,0,0,0,0,43,0,27,26,27,26,0,0,0,0,0,0,0,0,0,0,56,0,0,0,32,0,0,0,0,71,0,56,898,16,7.0,1.0


In [0]:
# FIND PEARSON CORRELATION COEFFICIENTS and P-VALUES

words = ['isolation', 'exclusion', 'left out', 'ostracism', 'lonely', 
            'loneliness', 'alone', 'depression', 'anxiety', 'bored', 
            'overwhelmed', 'angry', 'frustration', 'missing friends', 'stay in touch', 
            'worry', 'going out', 'go outside', 'self teach', 'hopeless', 
            'remote', 'segregated', 'discriminative', 'abandoned', 'tired', 
            'confused', 'separated', 'distant', 'disconnected', 'sad', 
            'stressed']


predictor_list = []
state_list = []
for i,state in enumerate(merged_df['state'].unique()):

    subset = merged_df[merged_df['state']==state]
    predictors = subset[['cases', 'cases_daily', 'deaths', 'deaths_daily']]
    word_cols = subset[words]

    for j,predictor in enumerate(predictors): 
        p_col = np.array(predictors[predictor])
        p_col_scaled = preprocessing.scale(p_col)
        corr_list = []
        pval_list = []
        for word_col in word_cols:
            w_col = np.array(word_cols[word_col])
            w_col_scaled = preprocessing.scale(w_col)
            corr, pval = pearsonr(p_col_scaled, w_col_scaled)
            corr_list.append(round(corr,4))
            pval_list.append(pval)
        corr_list = np.array(corr_list).reshape(1,len(words))
        pval_list = np.array(pval_list).reshape(1,len(words))
        corr_df = pd.DataFrame(data = corr_list, columns=words)
        pval_df = pd.DataFrame(data = pval_list, columns=words)
        
        state_list.append(state)
        predictor_list.append(predictor)
        if i==0 and j==0:
            states_corr_df = corr_df
            states_pval_df = pval_df
        else:
            states_corr_df = states_corr_df.append(corr_df)
            states_pval_df = states_pval_df.append(pval_df)

# states_pval_df = states_pval_df.fillna(0)
# states_pval_df = np.array(states_pval_df)

# create three masks
r1 = states_corr_df.applymap(lambda x: '{}*'.format(x))
r2 = states_corr_df.applymap(lambda x: '{}**'.format(x))
r3 = states_corr_df.applymap(lambda x: '{}***'.format(x))
# apply them where appropriate
states_corr_df = states_corr_df.mask(states_pval_df<=0.05,r1)
states_corr_df = states_corr_df.mask(states_pval_df<=0.01,r2)
states_corr_df = states_corr_df.mask(states_pval_df<=0.001,r3)

states_corr_df.insert(loc=0, column='state', value=state_list)
states_corr_df.insert(loc=1, column='predictor', value=predictor_list)
states_corr_df = states_corr_df.reset_index().drop(columns = 'index')

states_corr_df

#states_corr_df.to_csv('gdrive/My Drive/COVID-19/states_corr_df.csv')


/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/usr/local/lib/python3.6/dist-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation 

,state,predictor,isolation,exclusion,left out,ostracism,lonely,loneliness,alone,depression,anxiety,bored,overwhelmed,angry,frustration,missing friends,stay in touch,worry,going out,go outside,self teach,hopeless,remote,segregated,discriminative,abandoned,tired,confused,separated,distant,disconnected,sad,stressed
0,Alabama,cases,-0.5889***,0.0234,-0.0912,NaN,-0.0237,-0.0248,-0.1874,-0.4265***,-0.0248,-0.4035***,0.0025,-0.0312,0.1347,-0.0025,NaN,-0.2263*,0.0262,-0.1401,NaN,-0.0049,-0.0095,0.32**,-0.0878,-0.0237,-0.1208,-0.0283,0.0151,-0.0835,-0.1864,-0.2764*,0.0747
1,Alabama,cases_daily,-0.5238***,-0.043,0.0204,NaN,0.1023,0.0009,-0.1576,-0.3246**,-0.0177,-0.3081**,-0.0808,-0.0288,0.0634,0.0071,NaN,-0.2809*,0.1104,-0.1246,NaN,-0.0062,-0.0631,0.2038,-0.1151,-0.0298,-0.1021,-0.0145,-0.0662,-0.0561,-0.1222,-0.1812,0.0544
2,Alabama,deaths,-0.5808***,0.013,-0.1084,NaN,-0.0463,-0.0304,-0.1922,-0.4383***,-0.0428,-0.4028***,0.009,-0.0289,0.1436,-0.0259,NaN,-0.2342*,0.0155,-0.1505,NaN,-0.0137,-0.0349,0.3187**,-0.0888,-0.0377,-0.1164,-0.0152,-0.0038,-0.0834,-0.1731,-0.2737*,0.067
3,Alabama,deaths_daily,-0.2466*,-0.0919,-0.0649,NaN,0.0659,0.0222,-0.1599,-0.0548,-0.1721,-0.2209*,0.0009,-0.1066,0.1398,0.1313,NaN,-0.1811,-0.0787,-0.2207*,NaN,-0.1588,-0.1929,0.0539,-0.1217,-0.0996,-0.1525,0.041,0.0445,0.1813,0.0462,-0.2272*,-0.0206
4,Alaska,cases,-0.459***,0.0656,-0.324**,NaN,0.0427,-0.2589*,0.0547,-0.2056,0.0604,-0.1897,0.0163,-0.2049,NaN,NaN,NaN,0.1022,0.0186,-0.0082,NaN,-0.0014,-0.2564*,NaN,NaN,-0.1463,-0.0242,-0.069,-0.0221,-0.1828,0.0011,-0.0056,0.0457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Wisconsin,deaths_daily,-0.0057,-0.0285,-0.0414,-0.0824,0.0502,0.0082,-0.0429,-0.1192,-0.0987,0.0895,-0.1158,-0.0483,0.0816,-0.0727,-0.0142,-0.1997*,0.1218,0.0651,-0.0018,-0.0154,0.0307,0.1788,0.0284,0.0498,-0.0311,-0.1414,-0.173,0.1383,-0.0695,-0.2486**,-0.085
200,Wyoming,cases,-0.3814***,0.0265,0.0053,NaN,0.0336,0.1745,0.1293,-0.2341*,0.0564,0.0233,-0.0991,0.0311,0.1681,NaN,NaN,-0.0687,0.1085,-0.0373,NaN,-0.1694,0.0975,NaN,NaN,-0.082,0.1548,-0.1725,-0.0288,0.0789,-0.0101,0.0875,0.0117
201,Wyoming,cases_daily,0.0333,-0.0424,0.2011,NaN,-0.0726,0.044,0.0863,-0.02,-0.173,0.1573,0.0007,0.0448,0.0749,NaN,NaN,-0.09,0.0585,-0.0049,NaN,-0.0109,0.1977,NaN,NaN,-0.1085,-0.0938,0.0422,0.0891,-0.011,-0.117,-0.1044,-0.0576
202,Wyoming,deaths,-0.3867***,0.0041,0.0104,NaN,0.0456,0.2076,0.1567,-0.2542*,0.0986,-0.0456,-0.1205,0.0339,0.1837,NaN,NaN,-0.0609,0.1086,-0.0387,NaN,-0.181,0.1132,NaN,NaN,-0.0676,0.0723,-0.1277,-0.0273,0.0026,-0.0483,0.0818,0.0473


# National Level Data

In [0]:
national_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv')
national_df

,date,cases,deaths
0,2020-01-21,1,0
1,2020-01-22,1,0
2,2020-01-23,1,0
3,2020-01-24,2,0
4,2020-01-25,3,0
...,...,...,...
131,2020-05-31,1798764,104379
132,2020-06-01,1820658,105113
133,2020-06-02,1841442,106195
134,2020-06-03,1861419,107184


# County Level Data

In [0]:
# load in dataset from the New York Times
# https://github.com/nytimes/covid-19-data
# counts are cumulative values

county_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
county_df

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0
...,...,...,...,...,...,...
206588,2020-06-04,Sweetwater,Wyoming,56037.0,32,0
206589,2020-06-04,Teton,Wyoming,56039.0,100,1
206590,2020-06-04,Uinta,Wyoming,56041.0,13,0
206591,2020-06-04,Washakie,Wyoming,56043.0,38,3
